# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda

Vamos trabalhar com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [45]:
import patsy
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.model_selection import train_test_split

In [46]:
df = pd.read_csv('previsao_de_renda.csv')

In [47]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   id_cliente             15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12427 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  renda                  15000 non-null  float64
dtypes:

1. Ajuste um modelo para prever log(renda) considerando todas as covariáveis disponíveis.
    - Utilizando os recursos do Patsy, coloque as variáveis qualitativas como *dummies*.
    - Mantenha sempre a categoria mais frequente como casela de referência
    - Avalie os parâmetros e veja se parecem fazer sentido prático.      

In [48]:
# Criar a variável resposta logarítmica
df['log_renda'] = np.log(df['renda'])

# Usar o Patsy para transformar variáveis categóricas em dummies
# O "-1" no final da fórmula remove o intercepto para que possamos controlar manualmente as categorias de referência
f = 'log_renda ~ sexo + posse_de_veiculo + posse_de_imovel + qtd_filhos + tipo_renda + educacao + estado_civil + tipo_residencia + idade + tempo_emprego + qt_pessoas_residencia - 1'
y, X = patsy.dmatrices(f, df, return_type='dataframe')

# Verifique as primeiras linhas para ter certeza de que tudo foi codificado corretamente
print(X.head())

   sexo[F]  sexo[M]  posse_de_veiculo[T.True]  posse_de_imovel[T.True]  \
0      1.0      0.0                       0.0                      1.0   
1      0.0      1.0                       1.0                      1.0   
2      1.0      0.0                       1.0                      1.0   
3      1.0      0.0                       0.0                      1.0   
4      0.0      1.0                       1.0                      0.0   

   tipo_renda[T.Bolsista]  tipo_renda[T.Empresário]  \
0                     0.0                       1.0   
1                     0.0                       0.0   
2                     0.0                       1.0   
3                     0.0                       0.0   
4                     0.0                       0.0   

   tipo_renda[T.Pensionista]  tipo_renda[T.Servidor público]  \
0                        0.0                             0.0   
1                        0.0                             0.0   
2                        0.0    

In [49]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [50]:
model = sm.OLS(y_train, X_train).fit()

In [51]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:              log_renda   R-squared:                       0.355
Model:                            OLS   Adj. R-squared:                  0.353
Method:                 Least Squares   F-statistic:                     227.4
Date:                Sun, 20 Aug 2023   Prob (F-statistic):               0.00
Time:                        15:17:22   Log-Likelihood:                -10864.
No. Observations:                9941   AIC:                         2.178e+04
Df Residuals:                    9916   BIC:                         2.196e+04
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
sexo[F] 

2. Remova a variável menos significante e analise:
    - Observe os indicadores que vimos, e avalie se o modelo melhorou ou piorou na sua opinião.
    - Observe os parâmetros e veja se algum se alterou muito.  

In [52]:
# Ajuste a fórmula para excluir a variável tipo_residencia[T.Estúdio]
f = 'log_renda ~ sexo + posse_de_veiculo + posse_de_imovel + qtd_filhos + tipo_renda + educacao + estado_civil + idade + tempo_emprego + qt_pessoas_residencia - 1'

y, X = patsy.dmatrices(f, df, return_type='dataframe')

print(X.head())

   sexo[F]  sexo[M]  posse_de_veiculo[T.True]  posse_de_imovel[T.True]  \
0      1.0      0.0                       0.0                      1.0   
1      0.0      1.0                       1.0                      1.0   
2      1.0      0.0                       1.0                      1.0   
3      1.0      0.0                       0.0                      1.0   
4      0.0      1.0                       1.0                      0.0   

   tipo_renda[T.Bolsista]  tipo_renda[T.Empresário]  \
0                     0.0                       1.0   
1                     0.0                       0.0   
2                     0.0                       1.0   
3                     0.0                       0.0   
4                     0.0                       0.0   

   tipo_renda[T.Pensionista]  tipo_renda[T.Servidor público]  \
0                        0.0                             0.0   
1                        0.0                             0.0   
2                        0.0    

In [53]:
# Dividir os dados em conjunto de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Ajustar o modelo de regressão linear usando o conjunto de treinamento
model = sm.OLS(y_train, X_train).fit()

# Imprimir o resumo do modelo
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:              log_renda   R-squared:                       0.355
Model:                            OLS   Adj. R-squared:                  0.353
Method:                 Least Squares   F-statistic:                     287.1
Date:                Sun, 20 Aug 2023   Prob (F-statistic):               0.00
Time:                        15:17:34   Log-Likelihood:                -10866.
No. Observations:                9941   AIC:                         2.177e+04
Df Residuals:                    9921   BIC:                         2.192e+04
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
                                      coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
sexo[F]   

3. Siga removendo as variáveis menos significantes, sempre que o *p-value* for menor que 5%. Compare o modelo final com o inicial. Observe os indicadores e conclua se o modelo parece melhor. 

In [54]:
# Excluir as colunas tipo_renda, educacao e tipo_residencia do DataFrame df
cols_to_drop = ['tipo_renda', 'educacao', 'tipo_residencia']
df_filtered = df.drop(cols_to_drop, axis=1)

# Criar a variável resposta logarítmica
df_filtered['log_renda'] = np.log(df_filtered['renda'])

# Usar o Patsy para criar a matriz de design com as variáveis filtradas
f_new = 'log_renda ~ sexo + posse_de_veiculo + posse_de_imovel + qtd_filhos + estado_civil + idade + tempo_emprego + qt_pessoas_residencia - 1'
y_new, X_new = patsy.dmatrices(f_new, df_filtered, return_type='dataframe')

# Dividir os dados em conjunto de treinamento e teste
X_train_new, X_test_new, y_train_new, y_test_new = train_test_split(X_new, y_new, test_size=0.2, random_state=42)

# Ajustar o novo modelo
model_new = sm.OLS(y_train_new, X_train_new).fit()

# Imprimir o resumo do novo modelo
print(model_new.summary())


                            OLS Regression Results                            
Dep. Variable:              log_renda   R-squared:                       0.344
Model:                            OLS   Adj. R-squared:                  0.343
Method:                 Least Squares   F-statistic:                     472.7
Date:                Sun, 20 Aug 2023   Prob (F-statistic):               0.00
Time:                        15:17:38   Log-Likelihood:                -10950.
No. Observations:                9941   AIC:                         2.192e+04
Df Residuals:                    9929   BIC:                         2.201e+04
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
sexo[F]                 

> * <span style="color:green">Não houveram mudanças significativas entre o modelo inicial e o modelo final em relação a proporção da variabilidade na variável de resposta explicada pelas variáveis independentes, apesar disso devido ao modelo final ser um modelo mais simples, ele se torna mais fácil de se interpretar e menos suscetível a overfitting.</span>